# This sim is configured for HERMES pi+ data
### This can simply be run as a notebook
### What is fudge?
### Does deltakT = kT?
### Is every other parameter to be left the way it is?


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook 
#############################################

files = files = ['1000.xlsx','1001.xlsx','1002.xlsx','1003.xlsx','1004.xlsx','1005.xlsx',
         '1006.xlsx','1007.xlsx','1008.xlsx','1009.xlsx']
path = '~/GIT/collinearity/database/sidis/expdata/'
for F in files:
    if F == '1000.xlsx':
        data = pd.read_excel(path + F)
    else:
        data = pd.concat([data,pd.read_excel(path + F)],sort = True)
data = data.query("col == 'hermes' and hadron == 'pi+'")

In [2]:
class Rfilter(object):

    def __init__(self, hadron='pi+', fudge=[0, 0]):
        self.M = 0.938
        self.M2 = self.M**2
        self.set_Mh(hadron)
        

        self.fudge1 = fudge[0]
        self.fudge2 = fudge[1]

        self.MiT2 = 0.5 + 0.3 * fudge[0]
        self.MfT2 = 0.5 + 0.3 * fudge[1]
        self.kT = 0
    
        self.MX = 1.3
        self.Ma = 1.5
        self.Mb = 0.3
        self.deltaM = 0.3
        self.MJ = 0.3
        
        self.MiT = self.MiT2**0.5
        self.MfT = self.MfT2**0.5
        self.kT2 = self.kT**2
        
        # from paper, we need kf2, deltakT2, ki2
        # we need to have a way to vary them!
        self.kf       = 0
        self.ki       = 0
        self.kf2      = self.kf**2
        self.ki2      = self.ki**2
        self.deltakT  = 0
        self.deltakT2 = self.deltakT**2
    def set_ki(self,ki):
        self.ki = ki
        self.ki2 = ki**2
    def set_kf(self,kf):
        self.kf = kf
        self.kf2 = kf**2
    def set_kT(self,kT):
        self.kT = kT
        self.kT2 = kT**2
    def set_deltakT(self,deltakT):
        self.deltakT = deltakT
        self.deltakT2 = self.deltakT**2
    def set_Mh(self, hadron):
        if hadron == 'pi+':
            self.Mh = 0.135  #  
        if hadron == 'pi-':
            self.Mh = 0.135  #  
        if hadron == 'pi0':
            self.Mh = 0.135  #  
        if hadron == 'k+':
            self.Mh = 0.493
        if hadron == 'k-':
            self.Mh = 0.493
        if hadron == 'k0':
            self.Mh = 0.493
        self.Mh2 = self.Mh**2
    
    def get_ki2(self):
        return self.ki2
    def get_kf(self):
        return self.kf
    
    def get_W2(self, x, Q2):
        W2 = Q2 * (1. - x) / x + self.M2
        return W2

    def get_MBT(self, PhT):
        MBT = np.sqrt( self.Mh2 + PhT**2 )
        return MBT
        
    def get_MiT(self, x, Q2):
        xn = self.get_xn(x, Q2)
#    self.MiT = np.sqrt( (xn*self.kT**2 + xn*self.MX**2 - (1-xn)*xn*self.M2)/(1-xn) )
        self.MiT = np.sqrt((xn * self.kT**2 + xn * (self.Ma + self.Mb / np.sqrt(xn))
                            ** 2 - (1 - xn) * xn * self.M2 + self.deltaM**2 * self.fudge1) / (1 - xn))
        return self.MiT

    def get_MfT(self):
        self.MfT = np.sqrt(self.kT**2 + self.MJ**2 +
                           self.deltaM**2 * self.fudge2)
        return self.MfT

    def get_xn(self, x, Q2):
        return 2 * x / (1 + np.sqrt(1 + 4 * x**2 * self.M2 / Q2))

    def get_yh(self, x, z, Q2, PhT, hadron, sign=-1):
        xn = self.get_xn(x, Q2)
        self.set_Mh(hadron)
        expy = Q2**0.5 * z * (Q2 - xn**2 * self.M2)\
            / (2 * self.M2 * xn**2 * (self.Mh2 + PhT**2)**0.5)\
            + sign * Q2**0.5 / (xn * self.M) * (z**2 * (Q2 - xn**2 * self.M2)**2
                                        / (4 * self.M2 * xn**2 * (self.Mh2 + PhT**2)) - 1)**0.5
        return np.log(expy)


    # zn
    def get_zn(self, x, z, Q2, PhT, hadron):
        xn = self.get_xn(x,Q2)
        self.set_Mh(hadron)
        MBT = self.get_MBT(PhT)
        zn =  xn * z/ (2 * x) * ( 1. + np.sqrt( 1.- 4. * self.M2 * MBT**2 * x**2 /  ( Q2**2 * z**2) ) )
        return zn
        
# rapidity of the target

    def get_yp(self, x, Q2):
        xn = self.get_xn(x, Q2)
        return np.log(np.sqrt(Q2) / (xn * self.M))

    def get_yi(self, Q2):
        return 0.5 * np.log(Q2 / self.MiT**2)

    def get_yf(self, Q2):
        return -0.5 * np.log(Q2 / self.MfT**2)

    def get_MhT(self, PhT):
        return np.sqrt(self.Mh2 + PhT**2)

    def get_R(self, x, z, Q2, PhT, hadron):
        self.set_Mh(hadron)
        MfT = self.get_MfT()
        MiT = self.get_MiT(x, Q2)
        yi = self.get_yi(Q2)
        yf = self.get_yf(Q2)
        MhT = self.get_MhT(PhT)
        yh = self.get_yh(x, z, Q2, PhT, hadron)
        zn = self.get_zn(x, z, Q2, PhT, hadron)
        znhat = zn/z
        # from paper..
        Ph_kf = 0.5 * MhT * MfT * (np.exp(yf - yh) + np.exp(yh - yf)) - znhat/zn * PhT**2 - PhT * self.kT
        Ph_ki = 0.5 * MhT * MiT * (np.exp(yi - yh) - np.exp(yh - yi)) - PhT * self.kT
        return np.abs(Ph_kf / Ph_ki)

        
# R0, Eq. (4.14)...
    def get_R0(self, Q2):
        """
        Collinearity ratio defined in the paper Eq. (4.15)
        """
        return  np.maximum(np.maximum(self.ki2/Q2,self.kf2/Q2),self.kT2/Q2)  
        
        
# We call R1 in the new paper what was R in the previous...
    def get_R1(self, x, z, Q2, PhT, hadron):
        """
        Collinearity ratio defined in the paper Eq. (4.15)
        """
        return  self.get_R( x, z, Q2, PhT, hadron)  

# R2 from Eq. (4.17)        
    def get_R2(self, x, z, Q2, PhT, hadron):
        self.set_Mh(hadron)
        zn = self.get_zn(x, z, Q2, PhT, hadron)
        znhat = zn/z
        qT = -PhT/zn
        # from paper..
        return np.abs( -(1.-znhat) - znhat*qT**2/Q2 - (1.-znhat)*self.kf2/(znhat*Q2) -self.deltakT2/(znhat*Q2) + 2.*qT*self.deltakT/Q2 )

# Proximity is defined for a sphere of a selected radius(p) by the following function
def Proximity(data,domain,p,hadron,fudge):
    """
    data   = any Pandas DataFrame should do as long as it contains x,z,Q2, and pT
    domain = an array of values you wish to extract R0,R1,R2 
    p      = radius of sphere representing the demarcation of the proximity
    hadron = any hadron found in the data
    fudge  = idk what fudge is so I did the right thing and passed it along 
    """
    zeroProx = []
    proximity = []
    total = 0.
    Rval = Rfilter(hadron = hadron,fudge=fudge)
    for ki in domain:
        Rval.set_ki(ki)
        for kf in domain:
            Rval.set_kf(kf)
            for kT in domain:
                Rval.set_kT(kT)
                Rval.set_deltakT(kT)
                if total != 0:
                    proximity.append([ki,kf,kT,total/len(data)])
                    total = 0.
                else:
                    zeroProx.append([ki,kf,kT,total/len(data)])
                for x,z,Q2,PhT in zip(data.x,data.z,data.Q2,data.pT):
                    R0 = Rval.get_R0(Q2)
                    R1 = Rval.get_R1(x,z,Q2,PhT,hadron = hadron)
                    R2 = Rval.get_R2(x,z,Q2,PhT,hadron = hadron)
                    p2 = R0**2 + R1**2 + R2**2
          
                    if p2 < p**2:
                   
                        total += 1
    return zeroProx,proximity

In [3]:
domain = np.arange(0.05,0.81,0.01)
zeroProx, proximity = Proximity(data,domain,p=1,hadron = 'pi+',fudge = [0,0])
print max(proximity[:3])
# zeroProx is actually empty because the first iteration always stores [0.05, 0.05, 0.05, 0.0]
print zeroProx

NameError: global name 'deltakT' is not defined

In [ ]:
fig,ax = plt.subplots()
ax = Axes3D(fig)
proxDat = pd.DataFrame(proximity,columns = ['ki','kf','deltakT','proximity'])
proxDat.to_excel('~/GIT/collinearity/notebooks/SIDIS_Proximity/prox_sim_058001.xlsx')
ax.scatter(proxDat['ki'],proxDat['kf'],proxDat['deltakT'],c = proxDat['proximity'],cmap = "plasma")
ax.set_zlabel("$ki$")
ax.set_ylabel("$kf$")
ax.set_xlabel("$\delta kT$")

In [ ]:
fig,ax = plt.subplots()
ax.scatter(proxDat[2],proxDat[0],c = proxDat[3],cmap = 'plasma')
ax.set_ylabel("$ki$")
ax.set_xlabel("$\delta kT$")

In [ ]:
domain = np.linspace(.05,.8,num=15)
zeroProx, proximity = Proximity(data,domain,p=1,hadron = 'pi+',fudge = [0,0])
print max(proximity[:3])
print zeroProx

In [ ]:
fig,ax = plt.subplots()
ax = Axes3D(fig)
proxDat = pd.DataFrame(proximity,columns = ['ki','kf','deltakT','proximity'])
proxDat.to_excel('~/GIT/collinearity/notebooks/SIDIS_Proximity/prox_sim_lowres.xlsx')
ax.scatter(proxDat['ki'],proxDat['kf'],proxDat['deltakT'],c = proxDat['proximity'],cmap = "plasma")
ax.set_zlabel("$ki$")
ax.set_ylabel("$kf$")
ax.set_xlabel("$\delta kT$")

In [ ]:
fig,ax = plt.subplots()
ax.scatter(proxDat['deltakT'],proxDat['ki'],c = proxDat['proximity'],cmap = 'plasma')
ax.set_ylabel("$ki$")
ax.set_xlabel("$\delta kT$")

In [ ]:
fig,ax = plt.subplots()
ax.scatter(proxDat['kf'],proxDat['ki'],c = proxDat['proximity'],cmap = 'plasma')
ax.set_ylabel("$ki$")
ax.set_xlabel("$kf$")

In [ ]:
fig,ax = plt.subplots()
ax.scatter(proxDat['kf'],proxDat['deltakT'],c = proxDat['proximity'],cmap = 'plasma')
ax.set_ylabel("$\delta kT$")
ax.set_xlabel("$kf$")